# Tarea 2: NBA y PageRank

### Temas selectos de Ciencia de Datos
### Luis David Huante García
### 26 de septiembre de 2023

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Cargamos los datos con el nombre de archivo correcto
file_path_corrected = 'nba1415.csv'
nba_data_corrected = pd.read_csv(file_path_corrected)

#Mostramos datos corregidos
nba_data_corrected.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Date,NaN,Visitor/Neutral,PTS,Home/Neutral,PTS,NaN,NaN
1,Tue Oct 28 2014,Box Score,Houston Rockets,108,Los Angeles Lakers,90,NaN,NaN
2,Tue Oct 28 2014,Box Score,Orlando Magic,84,New Orleans Pelicans,101,NaN,NaN
3,Tue Oct 28 2014,Box Score,Dallas Mavericks,100,San Antonio Spurs,101,NaN,NaN
4,Wed Oct 29 2014,Box Score,Brooklyn Nets,105,Boston Celtics,121,NaN,NaN


Los datos tienen registros de partidos de basketball, con columnas para la fecha, los equipos visitantes y locales, y sus respectivos puntos. Las columnas no tienen un encabezado claro, por lo que es necesario limpiar los datos y prepararlos para construir el grafo.​​

In [ ]:
# Asignar la primera fila como encabezado del dataframe
nba_data_corrected.columns = nba_data_corrected.iloc[0]
nba_data = nba_data_corrected.drop(nba_data_corrected.index[0])

#Resultado
nba_data.head()


,Date,NaN,Visitor/Neutral,PTS,Home/Neutral,PTS,NaN,NaN
1,Tue Oct 28 2014,Box Score,Houston Rockets,108,Los Angeles Lakers,90,NaN,NaN
2,Tue Oct 28 2014,Box Score,Orlando Magic,84,New Orleans Pelicans,101,NaN,NaN
3,Tue Oct 28 2014,Box Score,Dallas Mavericks,100,San Antonio Spurs,101,NaN,NaN
4,Wed Oct 29 2014,Box Score,Brooklyn Nets,105,Boston Celtics,121,NaN,NaN
5,Wed Oct 29 2014,Box Score,Milwaukee Bucks,106,Charlotte Hornets,108,OT,NaN


In [ ]:
# Cambiamos el nombre de las columnas
nba_data.columns = ['Date', 'Box_Score', 'Visitor_Team', 'Visitor_Points', 'Home_Team', 'Home_Points', 'Extra1', 'Extra2']

# Eliminamos las columnas que innecesarias
nba_data_cleaned = nba_data.drop(columns=['Date', 'Box_Score', 'Extra1', 'Extra2'])

# Convertir las columnas de puntos a enteros
nba_data_cleaned['Visitor_Points'] = pd.to_numeric(nba_data_cleaned['Visitor_Points'])
nba_data_cleaned['Home_Points'] = pd.to_numeric(nba_data_cleaned['Home_Points'])

# Resultado
nba_data_cleaned.head()

,Visitor_Team,Visitor_Points,Home_Team,Home_Points
1,Houston Rockets,108,Los Angeles Lakers,90
2,Orlando Magic,84,New Orleans Pelicans,101
3,Dallas Mavericks,100,San Antonio Spurs,101
4,Brooklyn Nets,105,Boston Celtics,121
5,Milwaukee Bucks,106,Charlotte Hornets,108


In [ ]:
import networkx as nx

# Creamos un grafo dirigido
G = nx.DiGraph()

# Añadimos aristas con pesos. Si el equipo visitante gana, se añade un borde hacia el equipo local, y viceversa.
for index, row in nba_data_cleaned.iterrows():
    home_team = row['Home_Team']
    visitor_team = row['Visitor_Team']
    home_points = row['Home_Points']
    visitor_points = row['Visitor_Points']

    # Determinamos el  equipo ganador y el perdedor para establecer la dirección de la arista
    if home_points > visitor_points:
        winner, loser, weight = home_team, visitor_team, home_points - visitor_points
    else:
        winner, loser, weight = visitor_team, home_team, visitor_points - home_points

    # Añadimos los equipos como nodos y el partido como un borde dirigido
    G.add_node(home_team)
    G.add_node(visitor_team)
    G.add_edge(winner, loser, weight=weight)

#En este caso no se utilizan los pesos de las aristas, pero podría ser útil en otro tipo de tarea.

#Definimos el número de nodos y aristas
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

num_nodes, num_edges


(30, 684)

### Centralidad de Page Rank

In [ ]:
# Centralidad de PageRank
pagerank_scores = nx.pagerank(G)

# Ordenamos los equipos por su puntuación de PageRank
pagerank_sorted = sorted(pagerank_scores.items(), key=lambda item: item[1], reverse=True)

#10 equipos más poderosos según PageRank
top_10_pagerank = pagerank_sorted[:10]
top_10_pagerank_teams = [team for team, score in top_10_pagerank]

top_10_pagerank_teams


['New York Knicks',
 'Denver Nuggets',
 'Minnesota Timberwolves',
 'Detroit Pistons',
 'Charlotte Hornets',
 'Orlando Magic',
 'Washington Wizards',
 'Brooklyn Nets',
 'Los Angeles Lakers',
 'Philadelphia 76ers']

### Centralidad de grado

In [ ]:
# Calculamos la centralidad de grado para cada nodo.
degree_centrality_scores = nx.degree_centrality(G)

# Ordenamos los equipos por su centralidad de grado
degree_centrality_sorted = sorted(degree_centrality_scores.items(), key=lambda item: item[1], reverse=True)

#10 equipos más poderosos según la centralidad de grado
top_10_degree_centrality = degree_centrality_sorted[:10]
top_10_degree_centrality_teams = [team for team, score in top_10_degree_centrality]

top_10_degree_centrality_teams


['Chicago Bulls',
 'Cleveland Cavaliers',
 'Detroit Pistons',
 'San Antonio Spurs',
 'Boston Celtics',
 'Phoenix Suns',
 'Oklahoma City Thunder',
 'Milwaukee Bucks',
 'Denver Nuggets',
 'Utah Jazz']

## Conclusión

PageRank considera no solo la cantidad de partidos ganados (o la cantidad de aristas que apuntan hacia un nodo), sino también la calidad de esos partidos. Un equipo que gana contra equipos rankeados hasta arriba en el top de PageRank recibirá un mayor puntaje de PageRank que uno que gane contra equipos con puntajes bajos. Por esta razón, un equipo podría tener un alto PageRank incluso si no tiene el número más alto de victorias, pero ha ganado contra equipos importantes.

La centralidad de grado simplemente cuenta el número de victorias (aristas salientes) . Un equipo con muchas victorias tendrá una alta centralidad de grado, independientemente de la fuerza o el ranking de PageRank de los equipos contra los que ganó.

La presencia de equipos como los Detroit Pistons en ambas listas muestra un desempeño exitoso tanto en términos de cantidad de victorias como en la calidad de los oponentes vencidos. Por otro lado, equipos como los Chicago Bulls y los Cleveland Cavaliers, que aparecen en la lista de centralidad de grado pero no en la de PageRank, pueden haber ganado muchos partidos, pero contra oponentes con menores puntajes de PageRank. Esto puede deberse a la estructura del torneo o al azar en los emparejamientos de partidos.

Estas diferencias muestran cómo diferentes métricas pueden dar una imagen variada de lo que significa ser un "​equipo poderoso" en la liga. Mientras que la centralidad de grado ofrece una visión basada en la cantidad de victorias, ignorando contra quién se lograron, el PageRank proporciona una perspectiva más matizada que valora las victorias basándose en la calidad del oponente. Por lo tanto, un equipo puede tener un alto grado de centralidad por ganar muchos juegos, pero eso no necesariamente indica que pueden desempeñarse igual de bien contra equipos más fuertes, lo que sí se reflejaría en un alto PageRank.

Además, el PageRank puede ser particularmente útil en torneos donde hay un desequilibrio en la fuerza de los equipos, como puede ser común en ligas con una mezcla de equipos con diferentes niveles de habilidad y recursos. Por otro lado, la centralidad de grado es una medida más sencilla y directa que puede ser útil para obtener una rápida evaluación del desempeño sin considerar otros factores.
